## Projeto Regressão Logística
### CASE: 
#### Você foi contratado por uma das principais agências de marketing para ajudá-los com a rotatividade de clientes.  Essa agência tem muitos clientes que usam seu serviço específico que produz anúncios para os clientes ou sites de clientes, e eles notam que tem um bocado de clientes em excesso e atualmente selecionam aleatoriamente gerentes de contas para cada cliente.
#### Eles querem que você crie um modelo de aprendizado de máquina que ajudará a prever quais clientes irão parar de comprar seus produtos para que possam distribuir seus gerentes de compras de forma a ajudar os com o maior risco de deixar de usar seus produtos.
#### Crie um algoritmo que preveja quais dos futuros clientes têm maior probabilidade de abandono para que a empresa possa distribuir seus gerentes corretamente de acordo com os dados.

##### Colunas do arquivo:
- Name: name of the latest contact at company
- Age: customer age
- Total_Purchase: total ads purchased
- Account_Manager: 0 = no manager, 1 = account manager assigned
- Years: total years as a customer
- Num_Sites: number of websites that use the service
- Onboard_Date: date that the name of latest contact was onboarded
- Location: client address
- Company: name of client company
- Churn: 0 or 1 indicating whether customer has churned
---------------------------------------------------------------------

## Importação, tratamento e visualização dos dados:

In [116]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pyspark.sql.functions as F
from pyspark.ml.feature import VectorAssembler 

In [2]:
spark = SparkSession\
    .builder\
    .appName('Logistic Regression Project')\
    .getOrCreate()
    
spark

In [3]:
# Atribuição dos dados ao dataframe que será utilizado na criação do modelo

dados = spark\
    .read\
    .option('inferschema', 'true')\
    .option('header','true')\
    .csv('C:/Users/drumo/OneDrive/Documentos/Estudo/Programação/VSCode/Python/Datasets/customer_churn.csv')
dados.show(10) # Verificação da saída

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [4]:
dados.columns   # Verificação dos nomes

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
# Novos nomes que serão atribuídos às colunas

new_names = [
    'Nomes',
    'Idade',
    'Total_Comprado',
    'Gerente_Conta',
    'Anos',
    'Num_Sites',
    'Ultimo_Contato',
    'Localidade',
    'Companhia',
    'Abandonou']

In [6]:
dados = dados.toDF(* new_names)    # Atribuição dos novos nomes
dados.show(5)

+----------------+-----+--------------+-------------+----+---------+-------------------+--------------------+--------------------+---------+
|           Nomes|Idade|Total_Comprado|Gerente_Conta|Anos|Num_Sites|     Ultimo_Contato|          Localidade|           Companhia|Abandonou|
+----------------+-----+--------------+-------------+----+---------+-------------------+--------------------+--------------------+---------+
|Cameron Williams| 42.0|       11066.8|            0|7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|        1|
|   Kevin Mueller| 41.0|      11916.22|            0| 6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|        1|
|     Eric Lozano| 38.0|      12884.75|            0|6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|        1|
|   Phillip White| 42.0|       8010.76|            0|6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|        1|
|  Cynthia No

In [113]:
# Consulta das dez empresas que não abandonaram a agência com maior valor de gastos com a agência e o número de sites

dados.select('Nomes','Total_Comprado', 'Num_Sites').filter(F.col('Abandonou') == 0).orderBy('Total_Comprado', ascending=False).show(10)

+------------------+--------------+---------+
|             Nomes|Total_Comprado|Num_Sites|
+------------------+--------------+---------+
|     Ethan Cordova|      18026.01|      9.0|
|      Kevin Powell|      16955.76|      8.0|
|        Eric Terry|      16371.42|     10.0|
|      Holly Flores|      15878.11|      8.0|
|   Darin Alexander|      15858.91|      8.0|
|  Michael Williams|      15571.26|      9.0|
|     Kenneth James|      15516.52|     10.0|
|Catherine Johnston|      15509.97|      8.0|
|      Katie Wagner|      15423.03|      7.0|
|    Brandon Hunter|      15188.65|      8.0|
+------------------+--------------+---------+
only showing top 10 rows



In [114]:
# Consulta das dez empresas que abandonaram a agência com maior valor de gastos com a agência e o número de sites

dados.select('Nomes','Total_Comprado', 'Num_Sites').filter(F.col('Abandonou') == 1).orderBy('Total_Comprado', ascending=False).show(10)

+---------------+--------------+---------+
|          Nomes|Total_Comprado|Num_Sites|
+---------------+--------------+---------+
| Jason Jones MD|      16838.94|      8.0|
| Monique Medina|      15005.43|     12.0|
|Samantha Fisher|      14693.45|     10.0|
| Hailey Sanchez|      14429.41|     11.0|
|  Jacob Escobar|      14398.89|     12.0|
|  Kelly Carroll|      14383.36|     10.0|
|    Ashlee Carr|       14062.6|     11.0|
|   Shawn Chavez|      14036.28|     10.0|
|   Carla Howell|      13743.27|     11.0|
|  Kathryn Ponce|      13725.55|      9.0|
+---------------+--------------+---------+
only showing top 10 rows



#### Colunas de interesse que serão trabalhadas:
- Idade;
- Total_Comprado;
- Gerente_Conta;
- Anos;
- Num_Sites;
- Ultimo_Contato;
- Localidade;
- Abandonou

In [9]:
# Atribuição das colunas de interesse a um novo dataframe, no qual serão feitas as manipulações

dados_selec = dados.select([
    'Idade',
    'Total_Comprado',
    'Gerente_Conta',
    'Anos',
    'Num_Sites',
    'Ultimo_Contato',
    'Localidade',
    'Abandonou'])

dados_selec.show(10)

+-----+--------------+-------------+----+---------+-------------------+--------------------+---------+
|Idade|Total_Comprado|Gerente_Conta|Anos|Num_Sites|     Ultimo_Contato|          Localidade|Abandonou|
+-----+--------------+-------------+----+---------+-------------------+--------------------+---------+
| 42.0|       11066.8|            0|7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|        1|
| 41.0|      11916.22|            0| 6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|        1|
| 38.0|      12884.75|            0|6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|        1|
| 42.0|       8010.76|            0|6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|        1|
| 37.0|       9191.58|            0|5.56|      9.0|2016-01-19 15:31:15|765 Tricia Row Ka...|        1|
| 48.0|      10356.02|            0|5.12|      8.0|2009-03-03 23:13:37|6187 Olson Mounta...|        1|
| 44.0|      11331.58|            1|5.23|     11.0|2016-12-05 03:35:43|48

In [10]:
# Verificação da ocorrência de valores nulos no dataframe

for i in dados_selec.columns:
    k = 0
    k += dados_selec.filter(F.col(f"{i}").isNull()).count()

print(f'Número de valores nulos encontrados: {k}')

Número de valores nulos encontrados: 0


In [11]:
dados_selec.printSchema() # Verificação do esquema do dataframe

root
 |-- Idade: double (nullable = true)
 |-- Total_Comprado: double (nullable = true)
 |-- Gerente_Conta: integer (nullable = true)
 |-- Anos: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Ultimo_Contato: string (nullable = true)
 |-- Localidade: string (nullable = true)
 |-- Abandonou: integer (nullable = true)



In [61]:
# Criação da coluna "features", a qual será feita a partir da vetorização dos valores das demais colunas de input

assembler = VectorAssembler(
    inputCols=['Idade', 'Total_Comprado', 'Gerente_Conta', 'Anos', 'Num_Sites'],
    outputCol='features')

In [117]:
resultados = assembler.transform(dados_selec)
resultados.show(10)

+-----+--------------+-------------+----+---------+-------------------+--------------------+---------+--------------------+
|Idade|Total_Comprado|Gerente_Conta|Anos|Num_Sites|     Ultimo_Contato|          Localidade|Abandonou|            features|
+-----+--------------+-------------+----+---------+-------------------+--------------------+---------+--------------------+
| 42.0|       11066.8|            0|7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|        1|[42.0,11066.8,0.0...|
| 41.0|      11916.22|            0| 6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|        1|[41.0,11916.22,0....|
| 38.0|      12884.75|            0|6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|        1|[38.0,12884.75,0....|
| 42.0|       8010.76|            0|6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|        1|[42.0,8010.76,0.0...|
| 37.0|       9191.58|            0|5.56|      9.0|2016-01-19 15:31:15|765 Tricia Row Ka...|        1|[37.0,9191.58,0.0...|
| 48.0| 

In [64]:
# Atribuição das colunas necessárias ao modelo a um novo dataframe e sua visualização

dados_finais = resultados.select('features','Abandonou')
dados_finais.show()

+--------------------+---------+
|            features|Abandonou|
+--------------------+---------+
|[42.0,11066.8,0.0...|        1|
|[41.0,11916.22,0....|        1|
|[38.0,12884.75,0....|        1|
|[42.0,8010.76,0.0...|        1|
|[37.0,9191.58,0.0...|        1|
|[48.0,10356.02,0....|        1|
|[44.0,11331.58,1....|        1|
|[32.0,9885.12,1.0...|        1|
|[43.0,14062.6,1.0...|        1|
|[40.0,8066.94,1.0...|        1|
|[30.0,11575.37,1....|        1|
|[45.0,8771.02,1.0...|        1|
|[45.0,8988.67,1.0...|        1|
|[40.0,8283.32,1.0...|        1|
|[41.0,6569.87,1.0...|        1|
|[38.0,10494.82,1....|        1|
|[45.0,8213.41,1.0...|        1|
|[43.0,11226.88,0....|        1|
|[53.0,5515.09,0.0...|        1|
|[46.0,8046.4,1.0,...|        1|
+--------------------+---------+
only showing top 20 rows



In [67]:
dados_treino, dados_teste = dados_finais.randomSplit([0.75, 0.25]) # Separação dos valores de treino e teste (Treino: 75%; Teste:25%)

-----------------------------------------------
## Criação, treinamento e teste do modelo de regressão:

In [65]:
reg_log = LogisticRegression(labelCol='Abandonou') # Instanciamento do modelo de regressão logística


In [68]:
modelo_ajustado = reg_log.fit(dados_treino) # Treinamento do modelo

In [69]:
sumario_treinamento = modelo_ajustado.summary

In [74]:
sumario_treinamento.predictions.describe().show() # Verificação dos dados de treinamento para possíveis fins estatísticos

+-------+-------------------+-------------------+
|summary|          Abandonou|         prediction|
+-------+-------------------+-------------------+
|  count|                685|                685|
|   mean|0.17372262773722627|0.13138686131386862|
| stddev| 0.3791476422820743|0.33806981891127397|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [75]:
predicoes = modelo_ajustado.evaluate(dados_teste) # Teste do modelo

In [79]:
predicoes.predictions.show()

+--------------------+---------+--------------------+--------------------+----------+
|            features|Abandonou|       rawPrediction|         probability|prediction|
+--------------------+---------+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|        1|[0.53687730018387...|[0.63108569963774...|       0.0|
|[28.0,11128.95,1....|        0|[4.33108365093400...|[0.98701747665164...|       0.0|
|[28.0,11245.38,0....|        0|[3.74431169367081...|[0.97689458386493...|       0.0|
|[29.0,8688.17,1.0...|        1|[2.70053029922414...|[0.93705792843608...|       0.0|
|[29.0,11274.46,1....|        0|[4.67597842615127...|[0.99076958898738...|       0.0|
|[29.0,12711.15,0....|        0|[5.50168851635392...|[0.99593670107786...|       0.0|
|[30.0,8403.78,1.0...|        0|[6.11565456096835...|[0.99779683464916...|       0.0|
|[30.0,8874.83,0.0...|        0|[3.09202648526360...|[0.95656264457325...|       0.0|
|[31.0,5304.6,0.0,...|        0|[3.26257196892406...|[

In [80]:
# Avaliação binária do modelo obtido 
avaliacao = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Abandonou')

In [119]:
AUC = avaliacao.evaluate(predicoes.predictions) # AUC = 0.8061009817671809
print(f'Taxa de assertividade do modelo: {AUC*100:.2f}%')

Taxa de assertividade do modelo: 80.61%


---------------------------------------------------
## Verificando a assertividade do modelo com novos dados de entrada

In [91]:
modelo_final = reg_log.fit(dados_finais) # Treinamento do modelo a partir dos dados finais 

In [93]:
# Importação dos novos dados para teste

novos_clientes = spark\
    .read\
        .option('inferschema','true')\
        .option('header','true')\
        .csv('C:/Users/drumo/OneDrive/Documentos/Estudo/Programação/VSCode/Python/Datasets/new_customers.csv')
novos_clientes.show(10)

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [94]:
novos_clientes.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [101]:
new_names = [
    'Nomes',
    'Idade',
    'Total_Comprado',
    'Gerente_Conta',
    'Anos',
    'Num_Sites',
    'Ultimo_Contato',
    'Localidade',
    'Companhia'
]

In [102]:
# Atribuição dos novos nomes das colunas

novos_clientes = novos_clientes.toDF(*new_names)
novos_clientes.printSchema()

root
 |-- Nomes: string (nullable = true)
 |-- Idade: double (nullable = true)
 |-- Total_Comprado: double (nullable = true)
 |-- Gerente_Conta: integer (nullable = true)
 |-- Anos: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Ultimo_Contato: string (nullable = true)
 |-- Localidade: string (nullable = true)
 |-- Companhia: string (nullable = true)



In [103]:
teste_novos_clientes = assembler.transform(novos_clientes)

In [105]:
resultados_finais = modelo_final.transform(teste_novos_clientes) # Teste final da predição

In [106]:
resultados_finais.select('Companhia','prediction').show() # Verificação dos resultados da predição

+----------------+----------+
|       Companhia|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



## As empresas com maior probabilidade de abandonar a agência de marketing são:
### Cannon-Benson;
### Barron-Robertson;
### Sexton-Golden;
### Parks-Robbins.